# List of Packages to Download

```python
!pip install langchain-chroma
!pip install langchain
!pip install langchain_community
!pip install langchainhub
!pip install --upgrade --quiet  gpt4all > /dev/null

# 1. Get a Data Loader


In [1]:
from langchain_community.document_loaders import WebBaseLoader


In [2]:
loader = WebBaseLoader("https://www.foxsports.com/nba/lebron-james-player-game-log?season=2023&seasonType=reg")
data = loader.load()
data

[Document(page_content="\n\n\nLeBron James Game Log - NBA  | FOX Sports\n\n\n    my favs \n          Access and manage your favorites here\n          \n            DISMISS\n                  Home Scores Watch NFL Draft Podcasts Odds Super 6 Stories  \n                  Search\n                     \n                  Sign In\n                    \n                  Account\n                    \n              SPORTS & TEAMS\n             \n              PLAYERS\n             \n              SHOWS\n             \n              PERSONALITIES\n                    SPORTS   SPORTS & TEAMSPLAYERSSHOWSPERSONALITIES \n          SPORTS\n          \n                NFL\n                \n                NCAA FB\n                \n                MLB\n                \n                NBA\n                \n                NCAA BK\n                \n                NASCAR\n                \n                Soccer\n                \n                UFL\n                \n                NCAAW BK\n

# 2. Convert data to Vector Database


In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

In [4]:
Chroma.from_documents(documents=data, embedding=GPT4AllEmbeddings(), persist_directory="./chroma_db")

# 3. Make a RAG pipeline


In [5]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [21]:
llm = ChatOllama(model="llama3")
prompt = hub.pull("rlm/rag-prompt")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=GPT4AllEmbeddings())

#qa_chain = RetrievalQA.from_chain_type(


In [22]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [24]:
question = "Explain what the table is Showing"
result = qa_chain({"query": question })

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x2bdf9f510>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [9]:
result["result"]

"It appears that the text you provided is a table of basketball statistics for the player Giannis Antetokounmpo. The columns include:\n\n* Minutes Played (MIN)\n* Points (PTS)\n* Field Goals (FG)\n* Three Point Field Goals (3FG)\n* Free Throws (FT)\n* Offensive Rebounds (OFF REB)\n* Defensive Rebounds (DEF REB)\n* Total Rebounds (REB)\n* Assists (AST)\n* Steals (STL)\n* Blocks (BLK)\n* Personal Fouls (PF)\n* Turnovers (TO)\n* Plus/Minus (+/-)\n\nThe rows appear to represent different games or periods in the season, with each column showing the player's statistics for that particular game or period.\n\nPlease let me know if you have any further questions!"

In [10]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result["result"])

('It appears that the text you provided is a table of basketball statistics '
 'for the player Giannis Antetokounmpo. The columns include:\n'
 '\n'
 '* Minutes Played (MIN)\n'
 '* Points (PTS)\n'
 '* Field Goals (FG)\n'
 '* Three Point Field Goals (3FG)\n'
 '* Free Throws (FT)\n'
 '* Offensive Rebounds (OFF REB)\n'
 '* Defensive Rebounds (DEF REB)\n'
 '* Total Rebounds (REB)\n'
 '* Assists (AST)\n'
 '* Steals (STL)\n'
 '* Blocks (BLK)\n'
 '* Personal Fouls (PF)\n'
 '* Turnovers (TO)\n'
 '* Plus/Minus (+/-)\n'
 '\n'
 'The rows appear to represent different games or periods in the season, with '
 "each column showing the player's statistics for that particular game or "
 'period.\n'
 '\n'
 'Please let me know if you have any further questions!')
